# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
results = establishments.find(query)
print(results)

# Use count_documents to display the number of documents in the result
print("Number of documents in the result:", establishments.count_documents(query))
# Display the first document in the results using pprint
pprint(results[0])

Number of documents in the result: 0


IndexError: no such item for Cursor instance

In [7]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(f"There are a total of {len(df)} rows in the DataFrame\n")
# Display the first 10 rows of the DataFrame
df.head(10)

There are a total of 0 rows in the DataFrame



""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [12]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query_2 = {'LocalAuthorityName': {'$regex': 'London'}, 
          'RatingValue': {'$gte': '4'}
         }
new_results = establishments.find(query_2)


# Use count_documents to display the number of documents in the result
print(f"\nThere are a total of {establishments.count_documents(query_2)} documents in the results for London.\n")
# Display the first document in the results using pprint
pprint(establishments.find_one(query_2))


There are a total of 0 documents in the results for London.

None


In [13]:
# Convert the result to a Pandas DataFrame
df_2 = pd.DataFrame(new_results)
# Display the number of rows in the DataFrame
print(f"There are a total of {len(df_2)} rows in the DataFrame\n")
# Display the first 10 rows of the DataFrame
df_2.head(10)

There are a total of 0 rows in the DataFrame



""


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [14]:
pprint(establishments.find_one({'BusinessName': 'Penang Flavours'}, {'geocode.latitude', 'geocode.longitude'}))

{'_id': ObjectId('64265d0e021161ebabc2a04a'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384}}


In [18]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query_3 = {'RatingValue': '5', 
         'geocode.latitude': {'$lte': (latitude + degree_search), '$gte': (latitude - degree_search)}, 
         'geocode.longitude': {'$lte': (longitude + degree_search), '$gte': (longitude - degree_search)} 
         }
sort =  [('scores.Hygiene', 1)]

# Print the results
results_3 = list(establishments.find(query_3).sort(sort).limit(5))
pprint(results_3)

[]


In [19]:
# Convert result to Pandas DataFrame
dataframe = pd.DataFrame(results_3)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [21]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match = {'$match': {'scores.Hygiene': 0}}

# 2. Groups the matches by Local Authority
group = {'$group': {'_id': {'LocalAuthorityName': '$LocalAuthorityName'}, 'count': {'$sum': 1}}}
# 3. Sorts the matches from highest to lowest
sort = {'$sort': {'count': -1}}
# Print the number of documents in the result
pipeline = [match, group, sort]
pip_results = list(establishments.aggregate(pipeline))

# Print the first 10 results
pprint(pip_results[0:10])

[]


In [22]:
# Convert the result to a Pandas DataFrame
pip_result_df =  pd.json_normalize(pip_results)
# Display the number of rows in the DataFrame
print(f"There are {len(pip_result_df)} rows in the DataFrame.")
# Display the first 10 rows of the DataFrame
pip_result_df.head(10)

There are 0 rows in the DataFrame.


""
